In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

                                                      IMPORTING DATASET

In [6]:
df = pd.read_csv("/content/PS_20174392719_1491204439457_log.csv")
print(df.head())

   step      type    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815       170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295        21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145          181.0            0.00   
3     1  CASH_OUT    181.00   C840083671          181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  
0  M1979787155             0.0             0.0        0               0  
1  M2044282225             0.0             0.0        0               0  
2   C553264065             0.0             0.0        1               0  
3    C38997010         21182.0             0.0        1               0  
4  M1230701703             0.0             0.0        0               0  


In [7]:
print(df.tail())

         step      type      amount     nameOrig  oldbalanceOrg  \
6362615   743  CASH_OUT   339682.13   C786484425      339682.13   
6362616   743  TRANSFER  6311409.28  C1529008245     6311409.28   
6362617   743  CASH_OUT  6311409.28  C1162922333     6311409.28   
6362618   743  TRANSFER   850002.52  C1685995037      850002.52   
6362619   743  CASH_OUT   850002.52  C1280323807      850002.52   

         newbalanceOrig     nameDest  oldbalanceDest  newbalanceDest  isFraud  \
6362615             0.0   C776919290            0.00       339682.13        1   
6362616             0.0  C1881841831            0.00            0.00        1   
6362617             0.0  C1365125890        68488.84      6379898.11        1   
6362618             0.0  C2080388513            0.00            0.00        1   
6362619             0.0   C873221189      6510099.11      7360101.63        1   

         isFlaggedFraud  
6362615               0  
6362616               0  
6362617               0  
636261

In [9]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB
None


                                                                          CHECKING MISSING VALUES IN THE DATASET

In [10]:
missing_values = df.isnull().sum()
print("Missing Values in Each Column:\n", missing_values)

missing_percentage = (df.isnull().sum() / len(df)) * 100
print("\nPercentage of Missing Values in Each Column:\n", missing_percentage)

Missing Values in Each Column:
 step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

Percentage of Missing Values in Each Column:
 step              0.0
type              0.0
amount            0.0
nameOrig          0.0
oldbalanceOrg     0.0
newbalanceOrig    0.0
nameDest          0.0
oldbalanceDest    0.0
newbalanceDest    0.0
isFraud           0.0
isFlaggedFraud    0.0
dtype: float64


                                                                          CHECKING NO OF FRAUD AND NON FRAUD TRANSACTION

In [11]:
fraud_counts = df['isFraud'].value_counts()
print("Number of non-fraud transactions:", fraud_counts[0])
print("Number of fraud transactions:", fraud_counts[1])

Number of non-fraud transactions: 6354407
Number of fraud transactions: 8213


                                                                          NO OF VALUES IN EACH COLUMN

In [12]:
for column in df.columns:
  print(f"{column}: {df[column].nunique()}")

step: 743
type: 5
amount: 5316900
nameOrig: 6353307
oldbalanceOrg: 1845844
newbalanceOrig: 2682586
nameDest: 2722362
oldbalanceDest: 3614697
newbalanceDest: 3555499
isFraud: 2
isFlaggedFraud: 2


In [13]:
df.type.value_counts()

,count
type,
CASH_OUT,2237500
PAYMENT,2151495
CASH_IN,1399284
TRANSFER,532909
DEBIT,41432


                                                                          HANDLING ALL STRING VALUES IN MY DATASET

In [14]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 3.6 MB/s eta 0:00:00


In [16]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import category_encoders as ce

high_cardinality_columns = [col for col in df.select_dtypes(include='object').columns if df[col].nunique() > 10]
low_cardinality_columns = [col for col in df.select_dtypes(include='object').columns if df[col].nunique() <= 10]

label_encoder = LabelEncoder()
for col in high_cardinality_columns:
    df[col] = label_encoder.fit_transform(df[col])

data = pd.get_dummies(df, columns=low_cardinality_columns, drop_first=True)

target_encoder = ce.TargetEncoder(cols=high_cardinality_columns)
data[high_cardinality_columns] = target_encoder.fit_transform(data[high_cardinality_columns], data['isFraud'])

print("Encoding completed.")

Encoding completed.


In [17]:
print(df.dtypes)

step                int64
type               object
amount            float64
nameOrig            int64
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest            int64
oldbalanceDest    float64
newbalanceDest    float64
isFraud             int64
isFlaggedFraud      int64
dtype: object


In [18]:
print(data.dtypes)

step                int64
amount            float64
nameOrig            int64
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest            int64
oldbalanceDest    float64
newbalanceDest    float64
isFraud             int64
isFlaggedFraud      int64
type_CASH_OUT        bool
type_DEBIT           bool
type_PAYMENT         bool
type_TRANSFER        bool
dtype: object


In [19]:
non_numeric_columns = data.select_dtypes(include=['object']).columns
print(non_numeric_columns)

Index([], dtype='object')


                                                                        FEATURE SCALING USING STANDARDIZATION

In [20]:
numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns

scaler = StandardScaler()

data[numeric_columns] = scaler.fit_transform(data[numeric_columns])

print(data.head())

       step    amount  nameOrig  oldbalanceOrg  newbalanceOrig  nameDest  \
0 -1.703042 -0.281560 -1.318825      -0.229810       -0.237622  1.220490   
1 -1.703042 -0.294767 -0.538520      -0.281359       -0.285812  1.316232   
2 -1.703042 -0.297555 -1.185630      -0.288654       -0.292442 -0.408856   
3 -1.703042 -0.297555  1.445742      -0.288654       -0.292442 -0.472820   
4 -1.703042 -0.278532  0.146834      -0.274329       -0.282221  0.109953   

   oldbalanceDest  newbalanceDest    isFraud  isFlaggedFraud  type_CASH_OUT  \
0       -0.323814       -0.333411  -0.035951       -0.001586          False   
1       -0.323814       -0.333411  -0.035951       -0.001586          False   
2       -0.323814       -0.333411  27.815483       -0.001586          False   
3       -0.317582       -0.333411  27.815483       -0.001586           True   
4       -0.323814       -0.333411  -0.035951       -0.001586          False   

   type_DEBIT  type_PAYMENT  type_TRANSFER  
0       False          

In [23]:
!pip install imbalanced-learn

                                                                  SEPERATING THE DATASET FOR ANALYSIS

In [24]:
legit = df[df.isFraud == 0]
fraud = df[df.isFraud == 1]

In [25]:
print(legit.shape)
print(fraud.shape)

(6354407, 11)
(8213, 11)


In [26]:
pd.options.display.float_format = '{:.6f}'.format
print(legit.amount.describe())

count    6354407.000000
mean      178197.041727
std       596236.981347
min            0.010000
25%        13368.395000
50%        74684.720000
75%       208364.760000
max     92445516.640000
Name: amount, dtype: float64


In [27]:
pd.options.display.float_format = '{:.6f}'.format
print(fraud.amount.describe())

count       8213.000000
mean     1467967.299140
std      2404252.947240
min            0.000000
25%       127091.330000
50%       441423.440000
75%      1517771.480000
max     10000000.000000
Name: amount, dtype: float64


In [28]:
from imblearn.over_sampling import RandomOverSampler
import pandas as pd


X = df.drop(columns=['isFraud'])
y = df['isFraud']

oversampler = RandomOverSampler(sampling_strategy={1: y.value_counts()[0]}, random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X, y)


df_resampled = pd.concat([X_resampled, y_resampled], axis=1)


print("Counts after upscaling fraud transactions:")
print(df_resampled['isFraud'].value_counts())

Counts after upscaling fraud transactions:
isFraud
0    6354407
1    6354407
Name: count, dtype: int64


                                                                  SPLITTING FOR TRAINING AND TESTING

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

print("Training set class distribution:")
print(y_train.value_counts())
print("\nTesting set class distribution:")
print(y_test.value_counts())

Training set class distribution:
isFraud
1    5083526
0    5083525
Name: count, dtype: int64

Testing set class distribution:
isFraud
0    1270882
1    1270881
Name: count, dtype: int64


In [31]:
print(X_train.dtypes)

step                int64
type               object
amount            float64
nameOrig            int64
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest            int64
oldbalanceDest    float64
newbalanceDest    float64
isFlaggedFraud      int64
dtype: object


In [32]:
X = pd.get_dummies(df.drop(columns=['isFraud', 'isFlaggedFraud', 'nameOrig', 'nameDest']), columns=['type'], drop_first=True)
y = df['isFraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

                                                                  TRAINING A LOGISTIC REGRESSION MODEL

In [33]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

                                                                  CHECKING THE ACCURACY OF THE TRAINED MODEL

In [34]:
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.9991850841320085


In [35]:
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

Confusion Matrix:
 [[1270745     159]
 [    878     742]]


In [36]:
class_report = classification_report(y_test, y_pred)
print("Classification Report:\n", class_report)

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00   1270904
           1       0.82      0.46      0.59      1620

    accuracy                           1.00   1272524
   macro avg       0.91      0.73      0.79   1272524
weighted avg       1.00      1.00      1.00   1272524

